In [3]:
# ┌─────────────────────────────────────────────────────────┐
# │ Cell 1: TRAIN TabNet on DataTraining.xlsx and SAVE IT │
# └─────────────────────────────────────────────────────────┘

# (Run once)
# !pip install pytorch-tabnet pandas scikit-learn openpyxl joblib

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pytorch_tabnet.tab_model import TabNetClassifier
import joblib

# 1) Load and clean training data
df = pd.read_excel("DataTraining.xlsx")

# 2) Ensure numeric columns are numeric & fill missing with zero
num_cols = [
    "Gender", "Academic Percentage",
    "Analytical", "Logical", "Explaining",
    "Creative", "Detail-Oriented", "Helping",
    "Activity Preference", "Project Preference"
]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce").fillna(0)

# 3) Label-encode "Study Stream" and target "Degree Program"
encoders = {}
for col in ["Study Stream", "Degree Program"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

# 4) Split into features / target, dropping both University & Degree Program
X = df.drop(columns=["Degree Program", "University"]).values
y = df["Degree Program"].values

# 5) Train TabNetClassifier
clf = TabNetClassifier(seed=42, verbose=0)
clf.fit(
    X_train=X, y_train=y,
    eval_set=[(X, y)],
    eval_name=["train"],
    eval_metric=["accuracy"],
    max_epochs=100,
    patience=10,
    batch_size=64,
    virtual_batch_size=32
)

# 6) Save model + encoders
clf.save_model("tabnet_degree_model")
joblib.dump(encoders, "label_encoders.pkl")

print("✅ TabNet trained and saved.")



Early stopping occurred at epoch 30 with best_epoch = 20 and best_train_accuracy = 0.81052


C:\Users\Saad Mahmood\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_degree_model.zip
✅ TabNet trained and saved.


In [6]:
import pandas as pd
import numpy as np
import joblib
from pytorch_tabnet.tab_model import TabNetClassifier

# 1) Load TabNet model + encoders
clf = TabNetClassifier()
clf.load_model("tabnet_degree_model.zip")

encoders = joblib.load("label_encoders.pkl")
# encoders == {"Study Stream": LabelEncoder(), "Degree Program": LabelEncoder()}

# 2) Read and clean TestData.xlsx
test_df = pd.read_excel("TestData.xlsx")
numeric_fields = [
    "Gender", "Academic Percentage", "Analytical", "Logical", "Explaining",
    "Creative", "Detail-Oriented", "Helping", "Activity Preference", "Project Preference"
]
test_df[numeric_fields] = test_df[numeric_fields] \
    .apply(pd.to_numeric, errors="coerce") \
    .fillna(0)

# 3) Encode categorical input
test_df["Study Stream"] = encoders["Study Stream"].transform(test_df["Study Stream"])

# 4) Prepare features for prediction
X_test = test_df.drop(columns=["University", "Degree Program"], errors="ignore").values

# 5) Predict & decode
pred_idxs = clf.predict(X_test)
pred_labels = encoders["Degree Program"].inverse_transform(pred_idxs)

# 6) Build output df
output = test_df.copy()
output["Predicted Degree Program"] = pred_labels

# 7) If true labels exist, compute correctness + totals
has_truth = "Degree Program" in output.columns
if has_truth:
    # Decode true labels for display
    true_idxs = encoders["Degree Program"].transform(output["Degree Program"])
    true_labels = encoders["Degree Program"].inverse_transform(true_idxs)
    output["Degree Program"] = true_labels
    
    output["Result"] = np.where(
        output["Predicted Degree Program"] == output["Degree Program"],
        "Correct", "Incorrect"
    )
    total = len(output)
    correct = (output["Result"] == "Correct").sum()
    output["Total"] = total
    output["Percentage"] = (correct / total) * 100
else:
    output["Result"] = "N/A"
    output["Total"] = "N/A"
    output["Percentage"] = "N/A"

# 8) Dynamically assemble cols list in your desired order
base_cols = [
    "Gender", "Academic Percentage", "Study Stream",
    "Analytical", "Logical", "Explaining", "Creative",
    "Detail-Oriented", "Helping", "Activity Preference",
    "Project Preference", "Predicted Degree Program"
]
if has_truth:
    final_cols = base_cols + ["Degree Program", "Result", "Total", "Percentage"]
else:
    final_cols = base_cols + ["Result", "Total", "Percentage"]

# 9) Save
output[final_cols].to_excel("Result.xlsx", index=False)
print("✅ TabNet predictions saved to 'Result.xlsx'")


C:\Users\Saad Mahmood\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


✅ TabNet predictions saved to 'Result.xlsx'
